<a href="https://colab.research.google.com/github/chaeyoooo/capstondesign_voicefishing/blob/main/top10_voca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

from konlpy.tag import Okt
from collections import Counter
import pandas as pd

# ① 보이스피싱 STT 데이터 불러오기
phishing_df = pd.read_csv("/content/drive/MyDrive/unnormal_texts.csv")

# ② 전체 텍스트 합치기
all_text = " ".join(phishing_df['transcribed_text'])

# ③ 형태소 분석기 초기화
okt = Okt()

# ④ 명사 추출
nouns = okt.nouns(all_text)

print(f"✅ 명사 추출 완료: 총 {len(nouns)}개 단어")

# ⑤ 빈도수 계산
count = Counter(nouns)

# ⑥ 상위 100개 출력
top_words = count.most_common(100)

for i, (word, freq) in enumerate(top_words, 1):
    print(f"{i:>2}. {word:<15} {freq}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 34.9 MB/s eta 0:00:00
✅ 명사 추출 완료: 총 190935개 단어
 1. 네               16117
 2. 거               4215
 3. 본인              3918
 4. 지금              3745
 5. 제               3109
 6. 그               2724
 7. 이               2508
 8. 예               2464
 9. 저희              2425
10. 안               2107
11. 뭐               1931
12. 통장              1661
13. 부분              1560
14. 좀               1445
15. 고객              1418
16. 이제              1393
17. 확인              1328
18. 말씀              1322
19. 저               1243
20. 것               1203
21. 사건              1192
22. 때문              1179
23. 일단              1151
24. 원               1038
25. 수               1035
26. 사용              1021
27. 쪽               1019
28. 진행              929
29. 전화              927
30. 게               919
31. 은행              883
32. 계좌              856
33. 대해            

In [ ]:
import pandas as pd
from google.colab import drive

### ================================
### 1. 구글 드라이브 마운트
### ================================
drive.mount('/content/drive')

### ================================
### 2. 보이스피싱 STT 데이터 불러오기
### ================================
phishing_df = pd.read_csv("/content/drive/MyDrive/unnormal_texts.csv")

print(f"✅ STT 데이터 로드 완료: {phishing_df.shape[0]} rows")
print(phishing_df.head())

### ================================
### 3. 사용자 지정 keyword 리스트
### ================================
custom_keywords = [
    '통장', '고객', '은행', '계좌', '금융',
    '카드', '조사', '대출', '피해자', '연락',
    '자금', '피해', '수사', '진술', '불법'
]

print("✅ 사용자 지정 Keywords:", custom_keywords)

### ================================
### 4. keyword flag 컬럼 추가
### ================================
def has_keyword(text):
    return int(any(keyword in text for keyword in custom_keywords))

phishing_df['has_keyword'] = phishing_df['transcribed_text'].apply(has_keyword)

print("✅ keyword flag 컬럼 추가 완료")
print(phishing_df[['transcribed_text', 'has_keyword']].head())

### ================================
### 5. 감정분석 결과 merge
### ================================
emotion_df = pd.read_csv("/content/drive/MyDrive/emotion_results.csv")

merged_phishing = pd.merge(phishing_df, emotion_df, on='file', how='left')

print("✅ 감정분석 결과 merge 완료")
print(merged_phishing[['file', 'transcribed_text', 'emotion', 'confidence', 'has_keyword']].head())

### ================================
### 6. is_deepfake 컬럼 추가
### ================================
merged_phishing['is_deepfake'] = 0

print("✅ is_deepfake 컬럼 추가 완료")
print(merged_phishing[['file', 'transcribed_text', 'emotion', 'confidence', 'has_keyword', 'is_deepfake']].head())

### ================================
### 7. DataFrame 저장
### ================================
merged_phishing.to_csv("/content/drive/MyDrive/merged_phishing.csv", index=False)

print("✅ merged_phishing 저장 완료 → /content/drive/MyDrive/merged_phishing.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ STT 데이터 로드 완료: 407 rows
                                                file  \
0  _____________________________________________1...   
1  ____________________________________________tr...   
2  _______________________________OO_____________...   
3  _______________KB_____________________________...   
4  1_____________________________________________...   

                                    transcribed_text  
0   여보세요? 네 여기 강남서 사이버스사과예요 어디요? 강남서 사이버스사과요 강남서?...  
1   여보세요? 여기 강남서 사이버스 사과예요. 어디요? 강남서 사이버스 사과예요. 강...  
2   네. 네. 네. 네. 네. 네. 네. 네. 네. 네. 네. 네. 네. 네. 네. ...  
3   1단계로 알겠고요. 저희는 KB 금융괴로사 KB 저축은행인데요. 지금 5월에 저희...  
4   우리 햇살론 특성상 체무가 3개월 이상 쓰셔야 햇살론 진행이 가능하시거든요 3개월...  
✅ 사용자 지정 Keywords: ['통장', '고객', '은행', '계좌', '금융', '카드', '조사', '대출', '피해자', '연락', '자금', '피해', '수사', '진술', '불법']
✅ keyword flag 컬럼 추가 완료
                                    transcr